<a href="https://colab.research.google.com/github/fatemafaria142/Exploration-of-Large-Language-Models-in-Mental-Health-Advice-Generation/blob/main/ChatBot_Mental_Health_Advice_Generation_using_openchat_3_5_1210_and_LangChain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **Install Required Packages**

In [1]:
!pip install accelerate peft bitsandbytes transformers trl datasets torch langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.3/168.3 kB 16.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.9/150.9 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 25.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 803.6/803.6 kB 22.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.7/79.7 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 47.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 229.6/229.6 kB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━

#### **Dataset Link:** https://huggingface.co/datasets/Amod/mental_health_counseling_conversations

In [2]:
from datasets import load_dataset

# Load the dataset
instruct_tune_dataset = load_dataset("Amod/mental_health_counseling_conversations")

# Select 3000 rows for training
train_subset = instruct_tune_dataset["train"].select(indices=range(3000))

# Select the rest for testing
test_subset = instruct_tune_dataset["train"].select(indices=range(3000, len(instruct_tune_dataset["train"])))


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split: 0 examples [00:00, ? examples/s]

In [3]:
train_subset

Dataset({
    features: ['Context', 'Response'],
    num_rows: 3000
})

In [4]:
test_subset

Dataset({
    features: ['Context', 'Response'],
    num_rows: 512
})

In [5]:
# Check the sizes of the subsets
print("Train subset size:", len(train_subset))
print("Test subset size:", len(test_subset))

Train subset size: 3000
Test subset size: 512


In [6]:
# Show 5 examples from the train dataset
print("Train subset examples:")
for i in range(5):
    context = train_subset['Context'][i]
    response = train_subset['Response'][i]
    print(f"Example {i + 1}:")
    print(f"Context: {context}")
    print(f"Response: {response}")
    print("-------------------")

Train subset examples:
Example 1:
Context: I'm going through some things with my feelings and myself. I barely sleep and I do nothing but think about how I'm worthless and how I shouldn't be here.
   I've never tried or contemplated suicide. I've always wanted to fix my issues, but I never get around to it.
   How can I change my feeling of being worthless to everyone?
Response: If everyone thinks you're worthless, then maybe you need to find new people to hang out with.Seriously, the social context in which a person lives is a big influence in self-esteem.Otherwise, you can go round and round trying to understand why you're not worthless, then go back to the same crowd and be knocked down again.There are many inspirational messages you can find in social media.  Maybe read some of the ones which state that no person is worthless, and that everyone has a good purpose to their life.Also, since our culture is so saturated with the belief that if someone doesn't feel good about themselves

In [7]:
# Show 5 examples from the test dataset
print("\nTest subset examples:")
for i in range(5):
    context = test_subset['Context'][i]
    response = test_subset['Response'][i]
    print(f"Example {i + 1}:")
    print(f"Context: {context}")
    print(f"Response: {response}")
    print("-------------------")


Test subset examples:
Example 1:
Context: I had to put a restraining order against my ex-fiancé. He was served last night. He was mentally and verbally abusive towards me. I thought it was my nerves, but every time I stand up, I get sick. If I sit and rock, I'm fine.
Response: Hello, and thank you for your question. You may be right, your physical symptoms of getting sick to your stomach may very well be your nerves or anxiety. That is totally normal and understandable given your circumstances. Another thing that is not unusual is that you feel better when you sit and rock. That is actually a fairly common way that many people comfort themselves in times of high stress. If you are concerned that there is an actual physical illness causing these symptoms, you may want to visit your primary care provider. They may tell you that they don't see anything physically wrong with you, and at that point you may want to visit a counselor about the symptoms and your relationship experience. A vis

#### **Creating a Mental Health Support Prompt Using LangChain**

In [8]:
from langchain import PromptTemplate, LLMChain

def create_prompt(sample):
    # Create a sample prompt template
    template = """GPT4 Correct User: Put yourself in the shoes of a thoughtful mental health counselor. Imagine you're all set to help with kindness and understanding, ready to offer care and support. Your task is to extend understanding and guidance, providing support to individuals dealing with mental health challenges. Here is my mental health problems: {context}<|end_of_turn|>
GPT4 Correct Assistant: {response}<|end_of_turn|>
"""

    # Extract context and response from the sample
    example_context = sample['Context']
    example_response = sample['Response']

    # Create a prompt using the template and the example
    prompt_template = PromptTemplate(template=template, input_variables=["context", "response"])
    single_turn_prompt = prompt_template.format(context=example_context, response=example_response)

    return single_turn_prompt

#### **Generating LangChain Prompt for a Training Subset Examples**

In [9]:
prompt = create_prompt(train_subset[0])
print(prompt)

GPT4 Correct User: Put yourself in the shoes of a thoughtful mental health counselor. Imagine you're all set to help with kindness and understanding, ready to offer care and support. Your task is to extend understanding and guidance, providing support to individuals dealing with mental health challenges. Here is my mental health problems: I'm going through some things with my feelings and myself. I barely sleep and I do nothing but think about how I'm worthless and how I shouldn't be here.
   I've never tried or contemplated suicide. I've always wanted to fix my issues, but I never get around to it.
   How can I change my feeling of being worthless to everyone?<|end_of_turn|>
GPT4 Correct Assistant: If everyone thinks you're worthless, then maybe you need to find new people to hang out with.Seriously, the social context in which a person lives is a big influence in self-esteem.Otherwise, you can go round and round trying to understand why you're not worthless, then go back to the same 

In [10]:
prompt = create_prompt(train_subset[1])
print(prompt)

GPT4 Correct User: Put yourself in the shoes of a thoughtful mental health counselor. Imagine you're all set to help with kindness and understanding, ready to offer care and support. Your task is to extend understanding and guidance, providing support to individuals dealing with mental health challenges. Here is my mental health problems: I'm going through some things with my feelings and myself. I barely sleep and I do nothing but think about how I'm worthless and how I shouldn't be here.
   I've never tried or contemplated suicide. I've always wanted to fix my issues, but I never get around to it.
   How can I change my feeling of being worthless to everyone?<|end_of_turn|>
GPT4 Correct Assistant: Hello, and thank you for your question and seeking advice on this. Feelings of worthlessness is unfortunately common. In fact, most people, if not all, have felt this to some degree at some point in their life. You are not alone. Changing our feelings is like changing our thoughts - it's ha

In [11]:
prompt = create_prompt(train_subset[2])
print(prompt)

GPT4 Correct User: Put yourself in the shoes of a thoughtful mental health counselor. Imagine you're all set to help with kindness and understanding, ready to offer care and support. Your task is to extend understanding and guidance, providing support to individuals dealing with mental health challenges. Here is my mental health problems: I'm going through some things with my feelings and myself. I barely sleep and I do nothing but think about how I'm worthless and how I shouldn't be here.
   I've never tried or contemplated suicide. I've always wanted to fix my issues, but I never get around to it.
   How can I change my feeling of being worthless to everyone?<|end_of_turn|>
GPT4 Correct Assistant: First thing I'd suggest is getting the sleep you need or it will impact how you think and feel. I'd look at finding what is going well in your life and what you can be grateful for. I believe everyone has talents and wants to find their purpose in life. I think you can figure it out with so

#### **Generating LangChain Prompt for a Testing Subset Examples**

In [12]:
prompt = create_prompt(test_subset[0])
print(prompt)

GPT4 Correct User: Put yourself in the shoes of a thoughtful mental health counselor. Imagine you're all set to help with kindness and understanding, ready to offer care and support. Your task is to extend understanding and guidance, providing support to individuals dealing with mental health challenges. Here is my mental health problems: I had to put a restraining order against my ex-fiancé. He was served last night. He was mentally and verbally abusive towards me. I thought it was my nerves, but every time I stand up, I get sick. If I sit and rock, I'm fine.<|end_of_turn|>
GPT4 Correct Assistant: Hello, and thank you for your question. You may be right, your physical symptoms of getting sick to your stomach may very well be your nerves or anxiety. That is totally normal and understandable given your circumstances. Another thing that is not unusual is that you feel better when you sit and rock. That is actually a fairly common way that many people comfort themselves in times of high s

In [13]:
prompt = create_prompt(test_subset[1])
print(prompt)

GPT4 Correct User: Put yourself in the shoes of a thoughtful mental health counselor. Imagine you're all set to help with kindness and understanding, ready to offer care and support. Your task is to extend understanding and guidance, providing support to individuals dealing with mental health challenges. Here is my mental health problems: I’m trying to make marriage work after a split. Before our split, he lied a lot and broke every promise to me. I don't think he cheated. Last month, I asked what women work with him, so he told me. Yesterday, I found out about a girl that he said he forgot about. Should I be upset?<|end_of_turn|>
GPT4 Correct Assistant: I would ask you first what made you give him another chance after he repeatedly lied and broke every promise to you?  I would imagine if he repeatedly lied to you that it will damage your ability to trust him now.  Is he in therapy? Does he recognize that he has a problem and is he trying to repair it?  Even if he truly forgot to tell 

In [14]:
prompt = create_prompt(test_subset[2])
print(prompt)

GPT4 Correct User: Put yourself in the shoes of a thoughtful mental health counselor. Imagine you're all set to help with kindness and understanding, ready to offer care and support. Your task is to extend understanding and guidance, providing support to individuals dealing with mental health challenges. Here is my mental health problems: My fiancé and I broke up. He cheated on me numerous times. I kept forgiving but questioning his every move. He got tired and left.<|end_of_turn|>
GPT4 Correct Assistant: if he as cheated on you multiple times it is not healthy for you to continue seeing him.  However It takes time to heal your pain. You are not a robot that can just switch off your emotions.  Please surround yourself with people who can support and empower you. <|end_of_turn|>



In [ ]:
prompt = create_prompt(test_subset[2])
print(prompt)

### **Initializing the Model**
* Load the model using a 4-bit configuration, employing double quantization, and set bfloat16 as the compute data type.

* Notably, we opt for the instruct-tuned model in this instance rather than the base model. It's worth mentioning that fine-tuning a base model necessitates a more substantial amount of data!

In [15]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch

bnb_config = BitsAndBytesConfig(
        load_in_4bit=True, bnb_4bit_quant_type="nf4", bnb_4bit_compute_dtype="float16", bnb_4bit_use_double_quant=True
    )


* **Model and Tokenizer:** https://huggingface.co/openchat/openchat-3.5-1210

In [16]:
mode_id = "openchat/openchat-3.5-1210"

In [17]:
model = AutoModelForCausalLM.from_pretrained(
        mode_id, quantization_config=bnb_config, device_map="auto", use_cache=False
    )

config.json:   0%|          | 0.00/623 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/179 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:381: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.5` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(


In [18]:
tokenizer = AutoTokenizer.from_pretrained("openchat/openchat-3.5-1210")
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/502 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


### **Let's example how well the model does at this task currently  (without training):**
* `do_sample:`This parameter controls whether to use sampling during text generation. When do_sample is set to True, the model samples words for the next token based on their probabilities. If set to False, the model uses greedy decoding and selects the token with the highest probability at each step.
* `pad_token_id:` This parameter represents the token ID used for padding in the tokenizer. In text generation, it's common to set the padding token to the end-of-sequence token (eos_token_id) to ensure that the generated text is complete.
* `top_k:` top_k controls the number of highest probability options considered during sampling. For each token, the model will limit the sampling to the top k tokens with the highest probabilities.
* **High top_k Value:** More deterministic and focused generation. The
model considers a larger number of top probability options, resulting in conservative and predictable outputs.
* **Low top_k Value:** More randomness in sampling. The model restricts the number of top probability options, leading to diverse and creative text generation, but it may result in less coherent or focused output.

* `top_p:` top_p is used in nucleus sampling, where the model considers the cumulative probability mass of the top tokens and stops sampling when it reaches a predefined threshold. It helps in preventing the model from being overly verbose and encourages diversity in the generated text.
* **High top_p Value:** Promotes diverse and varied text generation by considering a larger range of probable tokens. It's good for creating exploratory and varied outputs.
* **Low top_p Value:** Adopts a stricter threshold for selecting tokens, resulting in more conservative and focused text generation. This may lead to more deterministic outputs closely tied to the most probable tokens.
* `temperature:` temperature controls the level of randomness in the sampling process. A higher temperature (e.g., 1.0) increases randomness, making the generated text more diverse. Lower values (e.g., 0.8) make the sampling more deterministic, with the model focusing on higher probability tokens.

**Documentation Link:**
* https://python.langchain.com/docs/integrations/llms/huggingface_pipelines


In [19]:
from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline
from transformers import pipeline
from langchain.prompts import PromptTemplate

# Additional configuration parameters
do_sample = True
pad_token_id = tokenizer.eos_token_id
top_k = 50  # You can adjust this value based on your preference
top_p = 0.95  # You can adjust this value based on your preference
temperature = 0.8  # You can adjust this value based on your preference

pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=1024, do_sample=do_sample, pad_token_id=pad_token_id, top_k=top_k, top_p=top_p, temperature=temperature)
hf = HuggingFacePipeline(pipeline=pipe)

# Create a sample prompt template

template = """GPT4 Correct User: Put yourself in the shoes of a thoughtful mental health counselor. Imagine you're all set to help with kindness and understanding, ready to offer care and support. Your task is to extend understanding and guidance, providing support to individuals dealing with mental health challenges. Here is my mental health problems: {context}<|end_of_turn|>
GPT4 Correct Assistant: """
prompt = PromptTemplate(template=template, input_variables=["context"])


chain = prompt | hf

context = "I'm going through some things with my feelings and myself. I barely sleep and I do nothing but think about how I'm worthless and how I shouldn't be here.\n   I've never tried or contemplated suicide. I've always wanted to fix my issues, but I never get around to it. How can I change my feeling of being worthless to everyone?"

print(chain.invoke({"context": context}))

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1473: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(


1. Acknowledge your feelings: It's essential to recognize and validate your emotions. Remember, it's okay to feel worthless sometimes. You're not alone in feeling this way.

2. Seek professional help: Talk to a mental health professional, such as a therapist or counselor, who can provide you with personalized support and advice.

3. Establish a routine: Create a daily routine that includes healthy habits like exercise, eating well, and getting enough sleep. This can help stabilize your mood and provide a sense of control.

4. Challenge negative thoughts: When you find yourself thinking negative thoughts, try to challenge them by asking yourself if they're accurate or helpful. Replace them with positive affirmations.

5. Develop a support network: Reach out to friends, family, or support groups to help you feel less isolated. Surround yourself with people who can provide encouragement and understanding.

6. Set small goals: Break down your larger goals into smaller, manageable steps. Ac

### **Setting up the Training**
we will be using the `huggingface` and the `peft` library!

In [20]:
from peft import AutoPeftModelForCausalLM, LoraConfig, get_peft_model, prepare_model_for_kbit_training

peft_config = LoraConfig(r=8, lora_alpha=16, lora_dropout=0.05, bias="none", task_type="CAUSAL_LM")


* we need to prepare the model to be trained in 4bit so we will use the  **`prepare_model_for_kbit_training`** function from peft




In [21]:
model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, peft_config)

# **Training Hyperparameters**
The choice of hyperparameters is contingent upon the desired training duration. Pay special attention to the following key factors:

* `num_train_epochs/max_steps:` Dictates the number of iterations over the data. Exercise caution, as an excessive number may lead to overfitting!

* `learning_rate:` Governs the convergence speed of the model. Adjust this parameter judiciously for optimal results.

In [22]:
from transformers import TrainingArguments
output_model= "openchat_instruct_generation"
training_arguments = TrainingArguments(
        output_dir=output_model,
        per_device_train_batch_size=1,
        gradient_accumulation_steps=4,
        optim="paged_adamw_32bit",
        learning_rate=2e-4,
        lr_scheduler_type="cosine",
        save_strategy="epoch",
        logging_steps=10,
        num_train_epochs=1,
        max_steps=100,
        fp16=True,
)


### **Setting up the trainer**

`max_seq_length`: Context window size


In [23]:
from trl import SFTTrainer

max_seq_length = 1024

trainer = SFTTrainer(
  model=model,
  peft_config=peft_config,
  max_seq_length=max_seq_length,
  tokenizer=tokenizer,
  packing=True,
  formatting_func=create_prompt,  # This will apply the create_prompt mapping to all training and test dataset
  args=training_arguments,
  train_dataset=train_subset,      # Pass train_subset as train_dataset
  eval_dataset=test_subset         # Pass test_subset as eval_dataset
)


Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:310: UserWarning: You passed `packing=True` to the SFTTrainer, and you are training your model with `max_steps` strategy. The dataset will be iterated until the `max_steps` are reached.
  warnings.warn(


### **Training starts here**

In [24]:
trainer.train()

You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss
10,2.120000
20,1.828700
30,1.871400
40,1.899300
50,1.804500
60,1.613200
70,1.770300
80,1.876400
90,2.065600
100,1.894000


TrainOutput(global_step=100, training_loss=1.874354419708252, metrics={'train_runtime': 1729.5976, 'train_samples_per_second': 0.231, 'train_steps_per_second': 0.058, 'total_flos': 1.7483553570816e+16, 'train_loss': 1.874354419708252, 'epoch': 0.35})

### **Save the model**

In [25]:
trainer.save_model("openchat_instruct_generation")

In [26]:
merged_model = model.merge_and_unload()

/usr/local/lib/python3.10/dist-packages/peft/tuners/lora/bnb.py:229: UserWarning: Merge lora module to 4-bit linear may get different generations due to rounding errors.
  warnings.warn(


#### **Generate Answers After Training**

* `invoke:` call the chain on an input
* `batch:` call the chain on a list of inputs
#### **Documentation Link:**
* https://python.langchain.com/docs/expression_language/interface

# **Conversation Buffer memory**

In [44]:
# Create a sample prompt template
template = """GPT4 Correct User: Put yourself in the shoes of a thoughtful mental health counselor. Imagine you're all set to help with kindness and understanding, ready to offer care and support. Your task is to extend understanding and guidance, providing support to individuals dealing with mental health challenges. Here is my mental health problems: {context}<|end_of_turn|>
GPT4 Correct Assistant: """
prompt = PromptTemplate(template=template, input_variables=["context"])

In [45]:
from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline
from transformers import pipeline
from langchain.prompts import PromptTemplate
from langchain.memory import ConversationBufferMemory
from langchain.chains import LLMChain

pipeline = pipeline("text-generation", model=merged_model, tokenizer=tokenizer, max_new_tokens=1024, do_sample=do_sample, pad_token_id=pad_token_id, top_k=top_k, top_p=top_p, temperature=temperature)
llm = HuggingFacePipeline(pipeline=pipeline)

memory = ConversationBufferMemory()
chat_llm_chain = LLMChain(
    llm=llm,
    prompt=prompt,
    verbose=True,
    memory=memory,
)

#### **Example: 01**

In [46]:
context = "I'm going through some things with my feelings and myself. I barely sleep and I do nothing but think about how I'm worthless and how I shouldn't be here.\n   I've never tried or contemplated suicide. I've always wanted to fix my issues, but I never get around to it. How can I change my feeling of being worthless to everyone?"
chat_llm_chain.predict(context=context)



> Entering new LLMChain chain...
Prompt after formatting:
GPT4 Correct User: Put yourself in the shoes of a thoughtful mental health counselor. Imagine you're all set to help with kindness and understanding, ready to offer care and support. Your task is to extend understanding and guidance, providing support to individuals dealing with mental health challenges. Here is my mental health problems: I'm going through some things with my feelings and myself. I barely sleep and I do nothing but think about how I'm worthless and how I shouldn't be here.
   I've never tried or contemplated suicide. I've always wanted to fix my issues, but I never get around to it. How can I change my feeling of being worthless to everyone?<|end_of_turn|>
GPT4 Correct Assistant: 


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:61: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(



> Finished chain.


"\nIt's not unusual to feel like you're not good enough or to struggle with feeling worthless. However, it's important to remember that everyone has unique qualities and strengths. It's also important to understand that people can be cruel and judgemental, and their opinion of you isn't necessarily accurate. \n\nIf you're having thoughts of self-harm, it's important to seek professional help right away. Your thoughts and feelings are important and deserve attention. A mental health professional can help you understand what's going on and offer strategies to help you cope. \n\nAlso, try talking to someone close to you about your thoughts. Sometimes, just talking to someone can help you feel less alone and more supported. You are not alone."

#### **Example: 02**

In [53]:
context = "I'm 15 and my girlfriend is 14. Am I a pedophile because I'm attracted to her and she's under 18?"
chat_llm_chain.predict(context=context)



> Entering new LLMChain chain...
Prompt after formatting:
GPT4 Correct User: Put yourself in the shoes of a thoughtful mental health counselor. Imagine you're all set to help with kindness and understanding, ready to offer care and support. Your task is to extend understanding and guidance, providing support to individuals dealing with mental health challenges. Here is my mental health problems: I'm 15 and my girlfriend is 14. Am I a pedophile because I'm attracted to her and she's under 18?<|end_of_turn|>
GPT4 Correct Assistant: 


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:61: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(



> Finished chain.


"\nI understand that you're feeling confused and worried about your feelings for your girlfriend, who is under 18. It's important to know that having a romantic relationship with someone under 18 is illegal in many jurisdictions, and it's not healthy or safe to pursue a relationship with someone who is not legally an adult.\n\nAttracting someone younger than oneself is not in itself a sign of pedophilia. Pedophilia is a mental disorder in which an adult has intense and recurring sexual fantasies about children who are under 13 years old. This disorder is illegal, harmful, and unacceptable.\n\nIn your situation, it seems that you are attracted to your girlfriend because she is younger than you. This attraction may be due to the excitement and novelty of being in a relationship with someone younger than oneself. However, it's important to remember that you are both still young and that your relationship may be more about exploring new emotions and experiences than about a mature, long-te

#### **Example: 03**

In [54]:
context = "I'm always listening to my husband, but it feels like he never listens to me."
chat_llm_chain.predict(context=context)



> Entering new LLMChain chain...
Prompt after formatting:
GPT4 Correct User: Put yourself in the shoes of a thoughtful mental health counselor. Imagine you're all set to help with kindness and understanding, ready to offer care and support. Your task is to extend understanding and guidance, providing support to individuals dealing with mental health challenges. Here is my mental health problems: I'm always listening to my husband, but it feels like he never listens to me.<|end_of_turn|>
GPT4 Correct Assistant: 


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:61: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(



> Finished chain.


'\nIt\'s wonderful you\'re reaching out for support, and I\'m here to help. It sounds like you\'re experiencing a sense of feeling unheard and unsupported by your husband. This can be really challenging, as it can make you feel like you\'re not being valued or respected.\n\nIt can be helpful to think about this situation as a two-way street. Just as you\'re seeking to feel heard and supported, your husband likely wants to feel the same. So, in order to work through this, both of you will need to focus on improving communication and understanding.\n\nHere are some suggestions that might help you and your husband:\n\n1. Set aside dedicated time for open communication. This could be daily, or as frequently as you both need. Use this time to listen to each other and share your thoughts, feelings, and concerns.\n\n2. When you\'re speaking with each other, try to use "I" statements. This can help you focus on your own thoughts and feelings, rather than placing blame on the other person. For 

In [57]:
memory.load_memory_variables({})

{'history': 'Human: I\'m going through some things with my feelings and myself. I barely sleep and I do nothing but think about how I\'m worthless and how I shouldn\'t be here.\n   I\'ve never tried or contemplated suicide. I\'ve always wanted to fix my issues, but I never get around to it. How can I change my feeling of being worthless to everyone?\nAI: \nIt\'s not unusual to feel like you\'re not good enough or to struggle with feeling worthless. However, it\'s important to remember that everyone has unique qualities and strengths. It\'s also important to understand that people can be cruel and judgemental, and their opinion of you isn\'t necessarily accurate. \n\nIf you\'re having thoughts of self-harm, it\'s important to seek professional help right away. Your thoughts and feelings are important and deserve attention. A mental health professional can help you understand what\'s going on and offer strategies to help you cope. \n\nAlso, try talking to someone close to you about your 

In [58]:
print(memory.buffer)

Human: I'm going through some things with my feelings and myself. I barely sleep and I do nothing but think about how I'm worthless and how I shouldn't be here.
   I've never tried or contemplated suicide. I've always wanted to fix my issues, but I never get around to it. How can I change my feeling of being worthless to everyone?
AI: 
It's not unusual to feel like you're not good enough or to struggle with feeling worthless. However, it's important to remember that everyone has unique qualities and strengths. It's also important to understand that people can be cruel and judgemental, and their opinion of you isn't necessarily accurate. 

If you're having thoughts of self-harm, it's important to seek professional help right away. Your thoughts and feelings are important and deserve attention. A mental health professional can help you understand what's going on and offer strategies to help you cope. 

Also, try talking to someone close to you about your thoughts. Sometimes, just talking

# **Conversation Buffer Window Memory**
* ConversationBufferWindowMemory keeps a list of the interactions of the conversation over time. It only uses the last K interactions. This can be useful for keeping a sliding window of the most recent interactions, so the buffer does not get too large.

In [59]:
from langchain.prompts import PromptTemplate
# Create a sample prompt template
template = """GPT4 Correct User: Put yourself in the shoes of a thoughtful mental health counselor. Imagine you're all set to help with kindness and understanding, ready to offer care and support. Your task is to extend understanding and guidance, providing support to individuals dealing with mental health challenges. Here is my mental health problems: {context}<|end_of_turn|>
GPT4 Correct Assistant: """
prompt = PromptTemplate(template=template, input_variables=["context"])

In [60]:
from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline
from transformers import pipeline
from langchain.memory import ConversationBufferWindowMemory
from langchain.chains import LLMChain

pipeline = pipeline("text-generation", model=merged_model, tokenizer=tokenizer, max_new_tokens=1024, do_sample=do_sample, pad_token_id=pad_token_id, top_k=top_k, top_p=top_p, temperature=temperature)
llm = HuggingFacePipeline(pipeline=pipeline)

memory = ConversationBufferWindowMemory(k=2)
chat_llm_chain_window = LLMChain(
    llm=llm,
    prompt=prompt,
    verbose=True,
    memory=memory,
)

#### **Example: 01**

In [61]:
context = "I have three children: a biological daughter, a stepdaughter, and a stepson. I found out my stepdaughter overdosed today. My daughter blames me. I told her to stay out of it. I'm scared of all three kids now. They all blame me for their choices they made. All three have drug addictions. I had a stroke in 2013 so I need to depend on people from now on."
chat_llm_chain_window.predict(context=context)



> Entering new LLMChain chain...
Prompt after formatting:
GPT4 Correct User: Put yourself in the shoes of a thoughtful mental health counselor. Imagine you're all set to help with kindness and understanding, ready to offer care and support. Your task is to extend understanding and guidance, providing support to individuals dealing with mental health challenges. Here is my mental health problems: I have three children: a biological daughter, a stepdaughter, and a stepson. I found out my stepdaughter overdosed today. My daughter blames me. I told her to stay out of it. I'm scared of all three kids now. They all blame me for their choices they made. All three have drug addictions. I had a stroke in 2013 so I need to depend on people from now on.<|end_of_turn|>
GPT4 Correct Assistant: 


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:61: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(



> Finished chain.


"\nIt sounds like you're going through a very difficult and painful time. When faced with these types of challenges, it's important to remember that you are not alone. You have a support system and can reach out to friends, family, and professionals to help you through this tough time.\n\nFirst, I want to acknowledge the pain and fear you're experiencing. It's completely natural to feel scared when you're dealing with a situation like this, especially since you're in a position where you need to depend on others for help. It's important to recognize that you're going through a lot and to take care of yourself. Make sure you're eating well, getting enough sleep, and giving yourself time to relax and unwind.\n\nAs for your daughter blaming you for her sister's overdose, it's common for people to blame others when they're dealing with a lot of pain and stress. It's important to remember that addiction is a complex issue that isn't caused by one person's actions. It's also important to rem

#### **Example: 02**

In [62]:
context = "I have a friend that who I used to be in a relationship with. It was brief and turned into us being just good friends. I spent the weekend with him and  it upset my boyfriend. Was i wrong?"
chat_llm_chain_window.predict(context=context)




> Entering new LLMChain chain...
Prompt after formatting:
GPT4 Correct User: Put yourself in the shoes of a thoughtful mental health counselor. Imagine you're all set to help with kindness and understanding, ready to offer care and support. Your task is to extend understanding and guidance, providing support to individuals dealing with mental health challenges. Here is my mental health problems: I have a friend that who I used to be in a relationship with. It was brief and turned into us being just good friends. I spent the weekend with him and  it upset my boyfriend. Was i wrong?<|end_of_turn|>
GPT4 Correct Assistant: 


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:61: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(



> Finished chain.


"\nNo, you are not wrong to spend time with a friend, even if that friend used to be your romantic partner. Relationships do not end when they stop being romantic; friendships can still be an important part of our lives. Your boyfriend's feelings are important to consider, but it's also important to maintain your own friendships and to not let your boyfriend dictate who you can and cannot be friends with. If your boyfriend is feeling upset by your spending time with this friend, it would be helpful to have an open conversation with him about your feelings and how important this friend is to you. It's also possible that you may need to make a decision about whether this friendship is healthy for you. If your boyfriend continues to feel upset and you are feeling conflicted, you may want to consider seeking couples counseling to help you and your boyfriend navigate these feelings."

#### **Example: 03**

In [63]:
context = "I'm depressed. I have been for years. I hide it from everyone because I'm scared of the reactions I'll get. Last time I tried telling my parents, it was a huge argument about me being too young to be depressed (I’m a legal adult), calling me ungrateful, and telling me that if I can't handle things now, it's only going to get worse in the future (which is turning out to be true). It's exhausting pretending to be okay, and I don't know how much longer I can try. I'm just really tired, and sadly, I can't afford the help I need on my own."
chat_llm_chain_window.predict(context=context)




> Entering new LLMChain chain...
Prompt after formatting:
GPT4 Correct User: Put yourself in the shoes of a thoughtful mental health counselor. Imagine you're all set to help with kindness and understanding, ready to offer care and support. Your task is to extend understanding and guidance, providing support to individuals dealing with mental health challenges. Here is my mental health problems: I'm depressed. I have been for years. I hide it from everyone because I'm scared of the reactions I'll get. Last time I tried telling my parents, it was a huge argument about me being too young to be depressed (I’m a legal adult), calling me ungrateful, and telling me that if I can't handle things now, it's only going to get worse in the future (which is turning out to be true). It's exhausting pretending to be okay, and I don't know how much longer I can try. I'm just really tired, and sadly, I can't afford the help I need on my own.<|end_of_turn|>
GPT4 Correct Assistant: 


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:61: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(



> Finished chain.


'\nI\'m so sorry you\'re feeling this way. It\'s natural to feel tired and overwhelmed when you\'re dealing with depression. It sounds like you\'ve been trying to keep your struggles to yourself, but I wonder if there might be someone you trust who could help you start to open up about what you\'re going through. This might be a close friend, a mentor, a teacher, or another family member. Sometimes just having one person we can share our struggles with can help a lot.It\'s also important to know that there are many free resources available for people experiencing depression. Here are a few resources that might be helpful:1) Mental Health America\'s "Finding Mental Health Care" page: [https://www.mhanational.org/finding-mental-health-care](https://www.mhanational.org/finding-mental-health-care)2) National Institute of Mental Health\'s "Get Mental Health Help" page: [https://www.nimh.nih.gov/health/find-help/index.shtml](https://www.nimh.nih.gov/health/find-help/index.shtml)3) Crisis Tex

In [64]:
memory.load_memory_variables({})

{'history': 'Human: I have a friend that who I used to be in a relationship with. It was brief and turned into us being just good friends. I spent the weekend with him and  it upset my boyfriend. Was i wrong?\nAI: \nNo, you are not wrong to spend time with a friend, even if that friend used to be your romantic partner. Relationships do not end when they stop being romantic; friendships can still be an important part of our lives. Your boyfriend\'s feelings are important to consider, but it\'s also important to maintain your own friendships and to not let your boyfriend dictate who you can and cannot be friends with. If your boyfriend is feeling upset by your spending time with this friend, it would be helpful to have an open conversation with him about your feelings and how important this friend is to you. It\'s also possible that you may need to make a decision about whether this friendship is healthy for you. If your boyfriend continues to feel upset and you are feeling conflicted, y

In [65]:
print(memory.buffer)

Human: I have a friend that who I used to be in a relationship with. It was brief and turned into us being just good friends. I spent the weekend with him and  it upset my boyfriend. Was i wrong?
AI: 
No, you are not wrong to spend time with a friend, even if that friend used to be your romantic partner. Relationships do not end when they stop being romantic; friendships can still be an important part of our lives. Your boyfriend's feelings are important to consider, but it's also important to maintain your own friendships and to not let your boyfriend dictate who you can and cannot be friends with. If your boyfriend is feeling upset by your spending time with this friend, it would be helpful to have an open conversation with him about your feelings and how important this friend is to you. It's also possible that you may need to make a decision about whether this friendship is healthy for you. If your boyfriend continues to feel upset and you are feeling conflicted, you may want to con